<a href="https://colab.research.google.com/github/Kolawole-a2/Kola_Projects/blob/main/Few_Practice_Code_for_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
from google.colab import files  # Only works in Google Colab

def alert_admin(event_type, log_entry):
    print(f"[ALERT] Event '{event_type}' detected! Log entry: {log_entry.strip()}")

def analyze_log(file_obj, pattern1, pattern2):
    event_pattern1 = re.compile(pattern1, re.IGNORECASE)
    event_pattern2 = re.compile(pattern2, re.IGNORECASE)

    for line in file_obj:
        decoded_line = line.decode('utf-8')
        if event_pattern1.search(decoded_line):
            alert_admin(pattern1, decoded_line)
        if event_pattern2.search(decoded_line):
            alert_admin(pattern2, decoded_line)

def main():
    print("Welcome to the Log Analyzer\n")
    pattern1 = input("Enter regex pattern for Event Type 1 (e.g., LOGIN FAILURE): ").strip()
    pattern2 = input("Enter regex pattern for Event Type 2 (e.g., UNAUTHORIZED ACCESS): ").strip()

    print("\nPlease upload your log file:")
    uploaded = files.upload()

    if uploaded:
        filename = next(iter(uploaded))
        with open(filename, 'rb') as file_obj:
            print(f"\nAnalyzing uploaded file: {filename}\n")
            analyze_log(file_obj, pattern1, pattern2)
    else:
        print("No file uploaded.")

# Run the main function
main()


Welcome to the Log Analyzer

Enter regex pattern for Event Type 1 (e.g., LOGIN FAILURE): LOGIN FAILURE
Enter regex pattern for Event Type 2 (e.g., UNAUTHORIZED ACCESS): UNAUTHORIZED ACCESS

Please upload your log file:


Saving UNSW_NB15_traintest_backdoor.csv to UNSW_NB15_traintest_backdoor (1).csv

Analyzing uploaded file: UNSW_NB15_traintest_backdoor (1).csv



In [ ]:
import re
import io
import random
import pandas as pd
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# Common regex patterns for dropdown
common_patterns = {
    "Login Failure": r"login failure",
    "Unauthorized Access": r"unauthorized access",
    "Backdoor Attack": r"backdoor",
    "Date Pattern (YYYY-MM-DD)": r"\d{4}-\d{2}-\d{2}",
    "IP Address": r"\b(?:\d{1,3}\.){3}\d{1,3}\b",
    "User 'admin'": r"user\s*'admin'",
    "Custom Pattern": ""
}

def alert_admin(event_type, log_entry):
    print(f"[ALERT] Event '{event_type}' detected! Log entry: {log_entry.strip()}")

def test_regex_on_lines(pattern, lines):
    try:
        regex = re.compile(pattern, re.IGNORECASE)
        print(f"\n🔍 Testing pattern: {pattern}")
        for i, line in enumerate(lines):
            if regex.search(line):
                print(f"✅ Match on Line {i+1}: {line.strip()}")
            else:
                print(f"❌ No match on Line {i+1}: {line.strip()}")
    except re.error as e:
        print(f"⚠️ Invalid regex: {e}")

def analyze_log(lines, pattern1, pattern2):
    event_pattern1 = re.compile(pattern1, re.IGNORECASE)
    event_pattern2 = re.compile(pattern2, re.IGNORECASE)
    found = False

    for line in lines:
        if event_pattern1.search(line):
            alert_admin(pattern1, line)
            found = True
        if event_pattern2.search(line):
            alert_admin(pattern2, line)
            found = True

    if not found:
        print("\nNo matching events were found.")

def run_gui():
    print("📁 Please upload your log or CSV file:")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded.")
        return

    filename = next(iter(uploaded))
    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()

    # Sample 5 lines to show user
    print(f"\n📄 File '{filename}' loaded. Showing 5 sample lines for testing:")
    sample_lines = random.sample(lines, min(5, len(lines)))
    for i, line in enumerate(sample_lines):
        print(f"Line {i+1}: {line.strip()}")

    # Dropdowns and text inputs for patterns
    dropdown1 = widgets.Dropdown(
        options=list(common_patterns.keys()),
        value="Login Failure",
        description='Event 1:',
    )
    pattern1_input = widgets.Text(description='Pattern 1:', value=common_patterns[dropdown1.value])

    dropdown2 = widgets.Dropdown(
        options=list(common_patterns.keys()),
        value="Unauthorized Access",
        description='Event 2:',
    )
    pattern2_input = widgets.Text(description='Pattern 2:', value=common_patterns[dropdown2.value])

    def update_pattern1(change):
        pattern1_input.value = common_patterns[change.new]

    def update_pattern2(change):
        pattern2_input.value = common_patterns[change.new]

    dropdown1.observe(update_pattern1, names='value')
    dropdown2.observe(update_pattern2, names='value')

    button_test = widgets.Button(description="🧪 Test Patterns on Sample Lines")
    button_run = widgets.Button(description="🚀 Scan Full File")

    def on_test_click(b):
        clear_output(wait=True)
        display(dropdown1, pattern1_input, dropdown2, pattern2_input, button_test, button_run)
        print(f"\n🔎 Sample lines from '{filename}':")
        for i, line in enumerate(sample_lines):
            print(f"Line {i+1}: {line.strip()}")
        test_regex_on_lines(pattern1_input.value, sample_lines)
        test_regex_on_lines(pattern2_input.value, sample_lines)

    def on_run_click(b):
        clear_output(wait=True)
        print(f"📊 Analyzing '{filename}' with your patterns...\n")
        analyze_log(lines, pattern1_input.value, pattern2_input.value)

    button_test.on_click(on_test_click)
    button_run.on_click(on_run_click)

    display(dropdown1, pattern1_input, dropdown2, pattern2_input, button_test, button_run)

# Run the GUI-driven analyzer
run_gui()


In [ ]:
import hashlib
import json
import os
from google.colab import files
import io

HASH_DB = 'file_hashes.json'

def alert_admin(event_type, file_path):
    print(f"[ALERT] Event '{event_type}' detected! File changed: {file_path}")

def calculate_file_hash(file_path):
    sha256 = hashlib.sha256()
    try:
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(4096), b''):
                sha256.update(chunk)
        return sha256.hexdigest()
    except FileNotFoundError:
        return None

def load_hash_db():
    if os.path.exists(HASH_DB):
        with open(HASH_DB, 'r') as f:
            return json.load(f)
    return {}

def save_hash_db(db):
    with open(HASH_DB, 'w') as f:
        json.dump(db, f, indent=4)

def store_file_hashes_interactive():
    print("Upload files to store their hashes:")
    uploaded = files.upload()
    if not uploaded:
        print("No files uploaded.")
        return

    hash_db = load_hash_db()

    for filename in uploaded.keys():
        # Save uploaded file locally
        with open(filename, 'wb') as f:
            f.write(uploaded[filename])
        file_hash = calculate_file_hash(filename)
        if file_hash:
            hash_db[filename] = file_hash
            print(f"Stored hash for {filename}: {file_hash}")
        else:
            print(f"Error hashing file: {filename}")

    save_hash_db(hash_db)
    print("Hashes stored successfully.\n")

def check_file_integrity_interactive():
    hash_db = load_hash_db()
    if not hash_db:
        print("Hash database empty or missing. Please store hashes first.")
        return

    print("Checking file integrity...\n")
    for file_path, old_hash in hash_db.items():
        current_hash = calculate_file_hash(file_path)
        if current_hash is None:
            print(f"Warning: File missing: {file_path}")
        elif current_hash != old_hash:
            alert_admin("File Integrity Violation", file_path)
        else:
            print(f"No change detected: {file_path}")

# Run interactively:
print("Welcome to File Integrity Monitoring System")

while True:
    print("\nSelect an option:")
    print("1. Upload files and store hashes")
    print("2. Check file integrity")
    print("3. Exit")

    choice = input("Enter your choice (1/2/3): ").strip()
    if choice == '1':
        store_file_hashes_interactive()
    elif choice == '2':
        check_file_integrity_interactive()
    elif choice == '3':
        print("Exiting.")
        break
    else:
        print("Invalid choice. Please enter 1, 2, or 3.")


In [17]:
# Modify file content in place (simulate tampering)
filename = "test.txt"  # Replace with the uploaded file name
with open(filename, 'a') as f:
    f.write("\nThis is a simulated file change for testing.")
print(f"Modified {filename}.\nNow checking for integrity...")

# Check for changes
check_file_integrity_interactive()


Modified test.txt.
Now checking for integrity...
Checking file integrity...

No change detected: Log4Shell_Report.docx
No change detected: Log4Shell Simulation and Mitigation Report.pdf
No change detected: Log4Shell_Report (1).docx
No change detected: Log4Shell Simulation and Mitigation Report.docx
No change detected: Log4Shell Simulation and Mitigation Report (1).docx
